In [ ]:
!git clone https://github.com/jfzhang95/pytorch-video-recognition.git
%cd pytorch-video-recognition

Cloning into 'pytorch-video-recognition'...
remote: Enumerating objects: 157, done.
remote: Total 157 (delta 0), reused 0 (delta 0), pack-reused 157
Receiving objects: 100% (157/157), 4.07 MiB | 19.04 MiB/s, done.
Resolving deltas: 100% (79/79), done.
/content/pytorch-video-recognition


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
!cp "/content/drive/My Drive/c3d-pretrained.pth" "/content/pretrainedWeights/"

In [ ]:
!pip uninstall -y kaggle 
!pip install kaggle==1.5.6
%env KAGGLE_USERNAME = mahmoud23
%env KAGGLE_KEY = 9c696931b0d31aecbe8c229e81026f87
!kaggle competitions download -c computervisioncompetitionsc2020

In [ ]:
!cp "/content/drive/My Drive/Training_set.rar" "/content/Data/" 

In [ ]:
!unrar x /content/Data/Training_set.rar "/content/pytorch-video-recognition/Data"

In [ ]:
!pip install tensorboardX

     |████████████████████████████████| 317kB 3.4MB/s 


In [ ]:
p.to_csv()

In [ ]:
!python train.py

Device being used: cuda:0
Initializing weights from: /content/pytorch-video-recognition/run/run_10/models/C3D-ucf101_epoch-9.pth.tar...
Total params: 78.02M
Training model on ucf101 dataset...
Number of train videos: 301
Number of val videos: 77
Number of test videos: 126
  0% 0/126 [00:00<?, ?it/s]here
16
33
-----------
0
tensor([103], device='cuda:0')
  1% 1/126 [00:00<00:16,  7.43it/s]here
16
24
-----------
2
tensor([119], device='cuda:0')
  2% 2/126 [00:00<00:16,  7.60it/s]here
16
32
-----------
4
tensor([21], device='cuda:0')
  2% 3/126 [00:00<00:15,  7.71it/s]here
16
26
-----------
0
tensor([110], device='cuda:0')
  3% 4/126 [00:00<00:15,  8.02it/s]here
16
26
-----------
0
tensor([106], device='cuda:0')
  4% 5/126 [00:00<00:14,  8.34it/s]here
16
51
-----------
4
tensor([30], device='cuda:0')
  5% 6/126 [00:00<00:14,  8.39it/s]here
16
45
-----------
3
tensor([94], device='cuda:0')
  6% 7/126 [00:00<00:13,  8.53it/s]here
16
29
-----------
1
tensor([51], device='cuda:0')
  6% 8/126 

In [ ]:
import os
from sklearn.model_selection import train_test_split

import torch
import cv2
import numpy as np
from torch.utils.data import Dataset

resize_height = 128
resize_width = 171
crop_size = 112


def process_video(video, action_name, save_dir):
        # Initialize a VideoCapture object to read video data into a numpy array
        video_filename = video.split('.')[0]
        if not os.path.exists(os.path.join(save_dir, video_filename)):
            os.mkdir(os.path.join(save_dir, video_filename))

        capture = cv2.VideoCapture(os.path.join("/content/pytorch-video-recognition/Testing_set", video))

        frame_count = int(capture.get(cv2.CAP_PROP_FRAME_COUNT))
        frame_width = int(capture.get(cv2.CAP_PROP_FRAME_WIDTH))
        frame_height = int(capture.get(cv2.CAP_PROP_FRAME_HEIGHT))

        # Make sure splited video has at least 16 frames
        EXTRACT_FREQUENCY = 4
        if frame_count // EXTRACT_FREQUENCY <= 16:
            EXTRACT_FREQUENCY -= 1
            if frame_count // EXTRACT_FREQUENCY <= 16:
                EXTRACT_FREQUENCY -= 1
                if frame_count // EXTRACT_FREQUENCY <= 16:
                    EXTRACT_FREQUENCY -= 1

        count = 0
        i = 0
        retaining = True
        lastframe=""
        while (count < frame_count and retaining):
            retaining, frame = capture.read()
            if frame is None:
                continue

            if count % EXTRACT_FREQUENCY == 0:
                if (frame_height != resize_height) or (frame_width != resize_width):
                    frame = cv2.resize(frame, (resize_width, resize_height))
                lastframe = frame
                cv2.imwrite(filename=os.path.join(save_dir, video_filename, '0000{}.jpg'.format(str(i))), img=frame)
                i += 1
            count += 1
        while (i<17):
          cv2.imwrite(filename=os.path.join(save_dir, video_filename, '0000{}.jpg'.format(str(i))), img=lastframe)
          i += 1
        

        # Release the VideoCapture once it is no longer needed
        
        capture.release()

In [ ]:
for video in os.listdir("/content/pytorch-video-recognition/Testing_set"):
  process_video(video,"no","/content/pytorch-video-recognition/OutData/test")

In [ ]:
c=0
for i in sorted(os.listdir("/content/pytorch-video-recognition/OutData/test/Basketball")):
  if len(os.listdir("/content/pytorch-video-recognition/OutData/test/Basketball/"+i))==0:
    print("here")
    print(i)
    if i==".ipynb_checkpoints":
      !rm -r "/content/pytorch-video-recognition/OutData/test/Basketball/$i"


here
.ipynb_checkpoints


In [ ]:
labels =[]
for i in (os.listdir("/content/pytorch-video-recognition/OutData/test/Basketball")):
  labels.append(i)
label2index = {index: label for index, label in enumerate(sorted(set(labels)))}
        

In [ ]:
label2index

{0: 'test_image (1)',
 1: 'test_image (10)',
 2: 'test_image (100)',
 3: 'test_image (101)',
 4: 'test_image (102)',
 5: 'test_image (103)',
 6: 'test_image (104)',
 7: 'test_image (105)',
 8: 'test_image (106)',
 9: 'test_image (107)',
 10: 'test_image (108)',
 11: 'test_image (109)',
 12: 'test_image (11)',
 13: 'test_image (110)',
 14: 'test_image (111)',
 15: 'test_image (112)',
 16: 'test_image (113)',
 17: 'test_image (114)',
 18: 'test_image (115)',
 19: 'test_image (116)',
 20: 'test_image (117)',
 21: 'test_image (118)',
 22: 'test_image (119)',
 23: 'test_image (12)',
 24: 'test_image (120)',
 25: 'test_image (121)',
 26: 'test_image (122)',
 27: 'test_image (123)',
 28: 'test_image (124)',
 29: 'test_image (125)',
 30: 'test_image (126)',
 31: 'test_image (13)',
 32: 'test_image (14)',
 33: 'test_image (15)',
 34: 'test_image (16)',
 35: 'test_image (17)',
 36: 'test_image (18)',
 37: 'test_image (19)',
 38: 'test_image (2)',
 39: 'test_image (20)',
 40: 'test_image (21)',
 

In [ ]:
data = pd.read_csv("/content/submit.csv")

In [ ]:
data["name"]=[label2index[i] for i in data["name"]]

In [ ]:
data.to_csv("submit.csv",index=False)

In [ ]:
l=[]
for i in os.listdir("/content/pytorch-video-recognition/Data/Training"):
  l.append(i)

In [ ]:
sorted(l)

['Basketball', 'Diving', 'Jumping', 'Tennis', 'Walking']

In [ ]:
{label: index for index, label in enumerate(sorted(set(l)))}

{'Basketball': 0, 'Diving': 1, 'Jumping': 2, 'Tennis': 3, 'Walking': 4}

In [ ]:
0 == 2
1 == 0
2 == 1
3 == 3 

False

In [ ]:
data["class"]=data["class"].map({0:2,1:0,2:1,3:3,4:4})

In [ ]:
data1 = data.copy()

In [ ]:
data["name"]=data["name"]+".mpg"

In [ ]:
!zip -r "/content/drive/My Drive/content.zip" "/content/pytorch-video-recognition"

Streaming output truncated to the last 5000 lines.
  adding: content/pytorch-video-recognition/OutData/train/Tennis/v_tennis_15_07/00001.jpg (deflated 4%)
  adding: content/pytorch-video-recognition/OutData/train/Tennis/v_tennis_15_07/000016.jpg (deflated 4%)
  adding: content/pytorch-video-recognition/OutData/train/Tennis/v_tennis_15_07/000020.jpg (deflated 4%)
  adding: content/pytorch-video-recognition/OutData/train/Tennis/v_tennis_15_07/000022.jpg (deflated 4%)
  adding: content/pytorch-video-recognition/OutData/train/Tennis/v_tennis_15_07/000014.jpg (deflated 4%)
  adding: content/pytorch-video-recognition/OutData/train/Tennis/v_tennis_15_07/000012.jpg (deflated 4%)
  adding: content/pytorch-video-recognition/OutData/train/Tennis/v_tennis_15_07/00000.jpg (deflated 4%)
  adding: content/pytorch-video-recognition/OutData/train/Tennis/v_tennis_15_07/000019.jpg (deflated 4%)
  adding: content/pytorch-video-recognition/OutData/train/Tennis/v_tennis_15_07/000024.jpg (deflated 4%)
  addi

In [ ]:
!rm -r /content/pytorch-video-recognition/Data